In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Import and rearrange data

In [2]:
# Load data
bkgd = pd.read_hdf("data/preprocessed/bkgd.h5", key="bkgd")
vlq = pd.read_hdf("data/preprocessed/vlq.h5", key="vlq")
X_train = pd.concat([bkgd, vlq])
del bkgd, vlq

In [3]:
# Train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_train.drop(["Label"], axis=1), X_train["Label"], 
                                                    test_size=1/3, random_state=56)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=60)

In [4]:
# Save train data
X_train.to_hdf("data/classifier/train.h5", key="X")
y_train.to_hdf("data/classifier/train.h5", key="y")

# Save validation data
X_val.to_hdf("data/classifier/validation.h5", key="X")
y_val.to_hdf("data/classifier/validation.h5", key="y")

# Save test data
X_test.to_hdf("data/classifier/test.h5", key="X")
y_test.to_hdf("data/classifier/test.h5", key="y")

In [5]:
# Get data samples
train_samples, val_samples, test_samples = X_train["Sample"], X_val["Sample"], X_test["Sample"]

# Get data weights
train_weights, val_weights, test_weights = X_train["gen_weights"], X_val["gen_weights"], X_test["gen_weights"]

# Remove sample and weight columns
X_train.drop(["Sample", "gen_weights"], axis=1, inplace=True)
X_val.drop(["Sample", "gen_weights"], axis=1, inplace=True)
X_test.drop(["Sample", "gen_weights"], axis=1, inplace=True)

# Calculate class weights
class_weights = {
    0: 1,
    1: len(y_train[y_train==0]) / len(y_train[y_train==1])
}

# Create model

In [6]:
inputs = keras.Input(shape=(69,))
bn1 = keras.layers.BatchNormalization()(inputs)
fc1 = keras.layers.Dense(100, activation="relu")(bn1)
bn2 = keras.layers.BatchNormalization()(fc1)
d1 = keras.layers.Dropout(0.15)(bn2, training=True)
fc2 = keras.layers.Dense(100, activation="relu")(d1)
bn3 = keras.layers.BatchNormalization()(fc2)
d2 = keras.layers.Dropout(0.15)(bn3, training=True)
fc3 = keras.layers.Dense(100, activation="relu")(d2)
outputs = keras.layers.Dense(1, activation="sigmoid")(fc2)

model = keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="binary_crossentropy", 
              metrics=["accuracy", keras.metrics.AUC()])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 69)                276       
_________________________________________________________________
dense (Dense)                (None, 100)               7000      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [7]:
# Model name
name = "Hidden:100,100,100|BatchS:64|Dropout:0.15|BatchNorm|Adam"

# Callbacks

In [8]:
# Tensorboard
TB = keras.callbacks.TensorBoard("logs/" + name, write_images=True)

# Early Stopping
ES = keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, mode="min")

# Model Checkpoint
MC = keras.callbacks.ModelCheckpoint("models/" + name + ".h5", save_best_only=True, monitor="val_loss",
                                     mode="min")

# Reduce LR on Plateau
LR = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, mode="min", 
                                       min_lr=1e-6)

# Training the Model

In [9]:
model.fit(X_train.values, y_train.values, batch_size=64, epochs=500, callbacks=[TB, ES, MC, LR],
          validation_data=(X_val.values, y_val.values, val_weights.values), shuffle=True,
          sample_weight=train_weights.values, class_weight=class_weights)

Epoch 1/500
   1/4311 [..............................] - ETA: 0s - loss: 3.4283e-04 - accuracy: 0.4062 - auc: 0.6007WARNING:tensorflow:From /home/gilbertocunha/.local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
4311/4311 [==============================] - 20s 5ms/step - loss: 3.5526e-06 - accuracy: 0.8115 - auc: 0.9335 - val_loss: 6.0104e-07 - val_accuracy: 0.8288 - val_auc: 0.9498
Epoch 2/500
4311/4311 [==============================] - 8s 2ms/step - loss: 1.2022e-06 - accuracy: 0.8349 - auc: 0.9464 - val_loss: 6.1645e-07 - val_accuracy: 0.8139 - val_auc: 0.9435
Epoch 3/500
4311/4311 [==============================] - 8s 2ms/step - loss: 1.1117e-06 - accuracy: 0.8519 - auc: 0.9532 - val_loss: 5.4306e-07 - val_accuracy: 0.8130 - val_auc: 0.9541
Epoch 4/500
4311/4311 [=======================